In [31]:
#imports
import pandas as pd
import tensorflow
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Sequential, load_model
from keras import layers
from keras.metrics import AUC
from keras.callbacks import EarlyStopping
from keras.optimizer_v2.adam import Adam
import matplotlib.pyplot as plt
from pair import Pair

In [32]:
#Creating classifier labels from the closing price
def createLabels(returns: list[float], divisor: int = 1) -> list[int]:
    scalar = np.std(returns) / divisor
    return [int(ret / scalar) for ret in returns]

In [33]:
# feature creation
eurusd = Pair('EURUSD', key='q0tsF6PmO8kro7SlVt0S', mode='hourly')
eurusd.ts = pd.DataFrame(eurusd.ts)

shorter = 5
longer = 15

sma_long = [None] * longer
sma_short = [None] * shorter
std_long = [None] * longer
std_short = [None] * shorter

for i in range(len(eurusd.ts) - 5):
    sma_short.append(np.mean(eurusd.ts.close[i:i+5]))
    std_short.append(np.std(eurusd.ts.close[i:i+5]))

    if not i >= len(eurusd.ts) - 15:
        sma_long.append(np.mean(eurusd.ts.close[i:i+15]))
        std_long.append(np.std(eurusd.ts.close[i:i+15]))

eurusd.ts['SMA_short'] = sma_short
eurusd.ts['STD_short'] = std_short
eurusd.ts['SMA_long'] = sma_long
eurusd.ts['STD_long'] = std_long

df = eurusd.ts[['open', 'high', 'low', 'close', 'SMA_short', 'STD_short', 'SMA_long', 'STD_long']].iloc[15:,:].reset_index(drop=True)
features = df.iloc[:-1,:]
features

,open,high,low,close,SMA_short,STD_short,SMA_long,STD_long
0,1.08195,1.08216,1.08088,1.08124,1.081868,0.000424,1.081823,0.000554
1,1.08124,1.08244,1.07976,1.07986,1.081906,0.000351,1.081790,0.000572
2,1.07986,1.08081,1.07962,1.08054,1.081488,0.000886,1.081633,0.000735
3,1.08052,1.08172,1.08030,1.08100,1.081154,0.000865,1.081511,0.000754
4,1.08100,1.08206,1.08083,1.08182,1.080918,0.000698,1.081407,0.000708
...,...,...,...,...,...,...,...,...
475,1.09294,1.09388,1.09271,1.09296,1.093428,0.000425,1.093775,0.000421
476,1.09296,1.09298,1.09102,1.09109,1.093278,0.000432,1.093709,0.000463
477,1.09109,1.09197,1.09072,1.09197,1.092800,0.000952,1.093484,0.000764
478,1.09198,1.09275,1.09171,1.09214,1.092386,0.000752,1.093380,0.000852


In [35]:
# label creation
labels_no_arr = createLabels(eurusd.logrs, 2)
label_arr = tensorflow.keras.utils.to_categorical(labels_no_arr, dtype='int64')
labels = label_arr[15:]
labels

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [39]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=.15)

ct = ColumnTransformer([('only numeric', MinMaxScaler(), features.columns)], remainder='passthrough')
features_train = ct.fit_transform(features_train)
features_test = ct.fit_transform(features_test)
all_features = ct.fit_transform(df)


In [47]:
model: Sequential
loss = 'categorical_crossentropy'
metrics = ['accuracy']
build = 1
activ = 'relu'

def build_model(name: str = 'FXc', metrics = metrics, loss = loss):
    model = Sequential(name = name)
    model.add(layers.InputLayer(input_shape=features.shape[1],))
    model.add(layers.Dense(512, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(256, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(128, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(128, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(32, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(8, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(label_arr.shape[1], name='Output', activation='softmax'))
    model.compile(loss=loss, optimizer=Adam(learning_rate=0.002), metrics=metrics)
    print(f'Generating new model {model.name}')
    model.summary()
    return model

if 'model' not in dir() or build: model = build_model()
else: print('Loaded previous model:'); model.summary()

Generating new model FXc
Model: "FXc"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               4608      
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                                 
 dense_14 (Dense)            (None, 128)               32896     
                                                                 
 dense_15 (Dense)            (None, 128)               16512     
                                                                 
 dense_16 (Dense)            (None, 32)                4128      
                                                                 
 dense_17 (Dense)            (None, 8)                 264       
                                                                 
 Output (Dense)              (None, 11

In [51]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=50, verbose=1)
history = model.fit(features_train, labels_train, batch_size=3, epochs=300, verbose=1, validation_split=.2, callbacks=[es])

Epoch 1/300
109/109 [==============================] - 1s 8ms/step - loss: 1.6350 - accuracy: 0.4847 - val_loss: 1.5428 - val_accuracy: 0.5976
Epoch 2/300
109/109 [==============================] - 1s 9ms/step - loss: 1.6165 - accuracy: 0.4847 - val_loss: 1.5516 - val_accuracy: 0.5610
Epoch 3/300
109/109 [==============================] - 1s 8ms/step - loss: 1.6027 - accuracy: 0.4847 - val_loss: 1.6333 - val_accuracy: 0.5976
Epoch 4/300
109/109 [==============================] - 1s 7ms/step - loss: 1.6093 - accuracy: 0.4816 - val_loss: 1.5606 - val_accuracy: 0.5976
Epoch 5/300
109/109 [==============================] - 1s 6ms/step - loss: 1.6241 - accuracy: 0.4816 - val_loss: 1.7116 - val_accuracy: 0.5610
Epoch 6/300
109/109 [==============================] - 1s 8ms/step - loss: 1.5949 - accuracy: 0.4847 - val_loss: 1.6100 - val_accuracy: 0.5976
Epoch 7/300
109/109 [==============================] - 1s 8ms/step - loss: 1.5860 - accuracy: 0.4877 - val_loss: 1.7314 - val_accuracy: 0.5488

In [52]:
y_est = model.predict(features_test)
y_est = np.argmax(y_est, axis=1)
y_true = np.argmax(labels_test, axis=1)

In [53]:
print(classification_report(y_true, y_est))

              precision    recall  f1-score   support

           0       0.57      0.87      0.69        39
           1       0.40      0.25      0.31         8
           2       0.25      0.33      0.29         3
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         9

    accuracy                           0.51        72
   macro avg       0.12      0.15      0.13        72
weighted avg       0.36      0.51      0.42        72



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

In [55]:
print(confusion_matrix(y_true, y_est))

[[34  1  1  3  0  0  0  0  0  0]
 [ 6  2  0  0  0  0  0  0  0  0]
 [ 2  0  1  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0]
 [ 1  0  1  0  0  0  0  0  0  0]
 [ 0  1  1  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0]
 [ 8  1  0  0  0  0  0  0  0  0]]


In [ ]:
# bayesian adjustment